In [59]:
#! /usr/bin/env python
# -*- coding: utf-8 -*- 
# source = http://www.moe.gov.cn/s78/A03/ghs_left/s181/index.html

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re

list_file_name = []
list_date = []
list_url = []

def get_soup(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    html_soup = BeautifulSoup(driver.page_source,'lxml')
    driver.close()
    return html_soup

def parse(html_soup):
    all_elems_text = html_soup.find("div",class_ = "sj_cen").find_all("a")
    all_elems_a = html_soup.find("div",class_ = "sj_cen").find_all("a")
    all_elems_d = html_soup.find("div",class_ = "sj_cen").find_all("span")
    #print(all_elems_text)
    #print(all_elems_a)
    
    text_pattern = re.compile("[\u4E00-\u9FA5\\s]+")
    url_pattern = re.compile("\/srcsite.+")

    for (item,href,date) in zip(all_elems_text,all_elems_a,all_elems_d):
        href_raw = href.get("href")
        try:
            text = re.search(text_pattern,item.get_text()).group()
            urls = re.search(url_pattern,href_raw).group()
            list_file_name.append(text)
            list_url.append(urls)
            list_date.append(date.get_text())
        except AttributeError: ## it might contain other errors
            pass
            
        #list_url.append(href)
        #list_file_name.append()
    
url_first_layer = ["http://www.moe.gov.cn/s78/A03/ghs_left/s181/index.html"] 
for num in range(1,20):
    url = "http://www.moe.gov.cn/s78/A03/ghs_left/s181/index_" + str(num) + ".html"
    url_first_layer.append(url)

for url in url_first_layer[1:20]:
    soup = get_soup(url)
    parse(soup)
    
#get_soup()


'''
driver2 = webdriver.Chrome()
for num in range(1,2):
    url = "http://www.moe.gov.cn/s78/A03/ghs_left/s181/index_" + str(num) + ".html"
    print(url)
    driver2.get(url)
    html_soup = BeautifulSoup(driver2.page_source,'lxml')
    all_elems = html_soup.find_all("li",class_ = "index_bar_line4")
    time.sleep(3)
    #print(all_elems)
    #print(all_elems)
    #print(all_elems_a)
'''

TypeError: expected string or bytes-like object

In [71]:
'''
list_file_name.append(text)
list_url.append(urls)
list_date.append(date.get_text())
'''

table = pd.DataFrame([list_file_name,list_url,list_date])
table_origin = table.T

In [77]:
table_origin.columns = ["school_name","url","create_date"]

In [79]:
table_origin.to_excel("University.xlsx")

In [92]:
print(table_origin)

                             school_name  \
0                     教育部关于同意建立哈尔滨音乐学院的函   
1                      教育部关于同意建立浙江音乐学院的函   
2               教育部关于同意赣南师范学院更名为赣南师范大学的函   
3              教育部关于同意河南中医学院更名为河南中医药大学的函   
4                教育部关于同意广东药学院更名为广东药科大学的函   
5                教育部关于同意广东医学院更名为广东医科大学的函   
6           教育部关于同意上海应用技术学院更名为上海应用技术大学的函   
7               教育部关于同意中国计量学院更名为中国计量大学的函   
8                教育部关于同意徐州医学院更名为徐州医科大学的函   
9                教育部关于同意辽宁医学院更名为锦州医科大学的函   
10              教育部关于同意浙江海洋学院更名为浙江海洋大学的函   
11             教育部关于同意石家庄经济学院更名为河北地质大学的函   
12            教育部关于同意景德镇陶瓷学院更名为景德镇陶瓷大学的函   
13              教育部关于同意苏州科技学院更名为苏州科技大学的函   
14       教育部关于同意在廊坊东方职业技术学院基础上建立河北东方学院的函   
15             教育部关于同意邵阳医学高等专科学校并入邵阳学院的函   
16                      教育部关于同意建立厦门医学院的函   
17                       教育部关于同意建立亳州学院的函   
18                   教育部关于同意建立郑州工程技术学院的函   
19                      教育部关于同意建立河南工学院的函   
20                     教育部关于同意建立汉江师范学院的函   
21                   教育部关于同意建立河北

In [98]:
## regex:同意(.*?)更名为(.*?)的
## regex:同意建立(.*?)的
## regex:同意在(.*?)基础上建立(.*?)的
## 实际不止125笔改名的数据，由于发函标题只有两类，要么是建立学校，要么是学校改名，很多学校建校一般是有个基础
## http://www.moe.gov.cn/srcsite/A03/s181/201604/t20160401_236257.html
## 比如说同意在山西煤炭管理干部学院基础上建立山西能源学院
## 还有些不规范的文件标题，还需要挖网页内容中的

rename_list = []
new_university_list = []
rename_list_2 = []

for name in table_origin["school_name"]:
    pattern1 = re.compile("同意(.*?)更名为(.*?)的")
    pattern2 = re.compile("同意建立(.*?)的")
    pattern3 = re.compile("同意在(.*?)基础上建立(.*?)的")
    result1 = re.findall(pattern1,name)
    result2 = re.findall(pattern2,name)
    result3 = re.findall(pattern3,name)
    if result1 != []:
        rename_list.extend(result1)
    if result2 != []:
        new_university_list.extend(result2)
    if result3 != []:
        rename_list_2.extend(result3)

print("发函改名的学校：125笔")
print(rename_list)
print("-------------------")
print("新学校：47笔")
print(new_university_list)
print("-------------------")
print("在某类基础上建立的学校:130笔")
print(rename_list_2)

发函改名的学校：125笔
[('赣南师范学院', '赣南师范大学'), ('河南中医学院', '河南中医药大学'), ('广东药学院', '广东药科大学'), ('广东医学院', '广东医科大学'), ('上海应用技术学院', '上海应用技术大学'), ('中国计量学院', '中国计量大学'), ('徐州医学院', '徐州医科大学'), ('辽宁医学院', '锦州医科大学'), ('浙江海洋学院', '浙江海洋大学'), ('石家庄经济学院', '河北地质大学'), ('景德镇陶瓷学院', '景德镇陶瓷大学'), ('苏州科技学院', '苏州科技大学'), ('四川医科大学', '西南医科大学'), ('甘肃中医学院', '甘肃中医药大学'), ('兰州商学院', '兰州财经大学'), ('大理学院', '大理大学'), ('泸州医学院', '四川医科大学'), ('成都信息工程学院', '成都信息工程大学'), ('西藏民族学院', '西藏民族大学'), ('喀什师范学院', '喀什大学'), ('陕西中医学院', '陕西中医药大学'), ('贵阳医学院', '贵州医科大学'), ('山东万杰医学院', '齐鲁医药学院'), ('郑州华信学院', '郑州工业应用技术学院'), ('武汉长江工商学院', '武汉工商学院'), ('湛江师范学院', '岭南师范学院'), ('沈阳建筑工程学院', '沈阳建筑大学'), ('广东商学院', '广东财经大学'), ('四川外语学院', '四川外国语大学'), ('哈尔滨德强商务学院', '黑龙江财经学院'), ('大连外国语学院', '大连外国语大学'), ('广州医学院', '广州医科大学'), ('武汉工业学院', '武汉轻工大学'), ('吉林建筑工程学院', '吉林建筑大学'), ('安徽建筑工业学院', '安徽建筑大学'), ('天津城市建设学院', '天津城建大学'), ('长春师范学院', '长春师范大学'), ('黑龙江科技学院', '黑龙江科技大学'), ('安徽中医学院', '安徽中医药大学'), ('山东轻工业学院', '齐鲁工业大学'), ('华北水利水电学院', '华北水利水电大学'), ('温州医学院', '温州医科大学'), ('江西中医学院', '江西中医药大学'), ('漳州师范学院', 